In [1]:
# %% Cell 1: Imports
import os
import random
import cv2
import numpy as np
import h5py
from tqdm import tqdm


In [2]:
# %% Cell 2: Parameterdefinition
# Pfad zu deinem Ordner mit Kalibrierungsbildern
calibration_data_path = "Mapillary_Vistas/validation/images"  # Passe diesen Pfad an

# Pfad, wo die vorverarbeiteten Daten dauerhaft gespeichert werden sollen (Cache-Datei)
cache_file = "Kalibr_Cache/cache_file.h5"  # Passe diesen Pfad an

# Zielgröße für die Bilder: (Channels, Height, Width)
target_shape = (3, 520, 520)

# Maximale Anzahl an Bildern, die verarbeitet werden sollen
max_samples = 600

# Anzahl der Bilder pro Batch (zur Reduzierung des RAM-Verbrauchs)
batch_size = 50


In [3]:
# %% Cell 3: Funktion zur Vorverarbeitung und Speicherung der Kalibrierungsdaten
def process_and_save_calibration_data(calibration_data_path, target_shape, cache_file, max_samples, batch_size):
    """
    Liest bis zu max_samples Bilder aus dem Ordner calibration_data_path, verarbeitet diese in Batches,
    und speichert sie in einem HDF5-File cache_file.
    
    :param calibration_data_path: Ordnerpfad mit den Kalibrierungsbildern.
    :param target_shape: Zielgröße als (Channels, Height, Width), z. B. (3, 520, 520).
    :param cache_file: Pfad zur HDF5-Datei, in der die vorverarbeiteten Bilder dauerhaft gespeichert werden.
    :param max_samples: Maximale Anzahl an Bildern, die verarbeitet werden sollen.
    :param batch_size: Anzahl der Bilder, die pro Batch verarbeitet werden.
    """
    # Alle Bildpfade im Ordner sammeln
    all_images = [os.path.join(calibration_data_path, f)
                  for f in os.listdir(calibration_data_path)
                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Falls max_samples gesetzt ist, wähle zufällig bis zu max_samples Bilder aus
    if max_samples is not None:
        all_images = random.sample(all_images, min(len(all_images), max_samples))
    total_images = len(all_images)
    
    print(f"Verarbeite {total_images} Bilder in Batches à {batch_size} und speichere in '{cache_file}'...")
    
    # Erstelle das HDF5-File mit einem Dataset für alle Bilder:
    with h5py.File(cache_file, 'w') as hf:
        # Erstelle ein Dataset mit der Form (total_images, Channels, Height, Width)
        dset = hf.create_dataset("calibration_images", shape=(total_images,) + target_shape, dtype=np.float32)
        
        idx = 0
        # Verwende tqdm, um die Verarbeitung der Batches anzuzeigen
        for i in tqdm(range(0, total_images, batch_size), desc="Verarbeite Batches"):
            batch_paths = all_images[i:i+batch_size]
            batch_data = []
            for img_path in batch_paths:
                image = cv2.imread(img_path)
                if image is None:
                    continue
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # Zielgröße: target_shape ist (C, H, W) – also Resize auf (target_shape[2], target_shape[1])
                image = cv2.resize(image, (target_shape[2], target_shape[1]))
                image = image.astype(np.float32) / 255.0
                # Transponiere das Bild in die Form (C, H, W)
                image = np.transpose(image, (2, 0, 1))
                batch_data.append(image)
            if batch_data:
                batch_data = np.stack(batch_data, axis=0)
                dset[idx: idx + batch_data.shape[0]] = batch_data
                idx += batch_data.shape[0]
    print(f"Kalibrierungsdaten für {total_images} Bilder wurden in '{cache_file}' gespeichert.")


In [4]:
# %% Cell 4: Ausführung der Funktion
# Starte den Vorverarbeitungsprozess:
process_and_save_calibration_data(
    calibration_data_path=calibration_data_path,
    target_shape=target_shape,
    cache_file=cache_file,
    max_samples=max_samples,
    batch_size=batch_size
)


Verarbeite 600 Bilder in Batches à 50 und speichere in 'Kalibr_Cache/cache_file.h5'...


Verarbeite Batches: 100%|██████████| 12/12 [00:48<00:00,  4.02s/it]


Kalibrierungsdaten für 600 Bilder wurden in 'Kalibr_Cache/cache_file.h5' gespeichert.
